# Move PoV (RFID antennas) from one fixture to another

This module displays RFID PoV parameter.

Module interaction:

    Environement selection
    Premise selection
    PoV selection in the premise


## Select an environment

In [ ]:
%load_ext autoreload
%autoreload 2
from libConfUtils import chooseEnv as chooseEnv
from libConfUtils import deviceUtils as deviceUtils
from libConfUtils import povUtils as povUtils
from libConfUtils import uiUtils as uiUtils
from libConfUtils import premiseNhubUtils as premiseNhubUtils
from libConfUtils import settingsUtils as settingsUtil

Select the Premise where is located the device.

In [ ]:
#%%
status = True
myEnv = chooseEnv.choosePremise()

In [ ]:
#%% display choice
if not myEnv.get('PremiseCode'):
    text = 'No Premise Found '
    uiUtils.displayMessage('Device search',text)
    print(text)
    status = False
else:  
    print("Organization: " + myEnv.get('urlDP') + " Premise code: "+  myEnv.get('PremiseCode'))

Load all the povs linked to the premise

In [ ]:
if status:
    listOfPov = povUtils.getPovOfPremise(myEnv)
    if not listOfPov:
        text = "Premise: " + str(myEnv.get('PremiseCode')) + ' No PoV found '
        uiUtils.displayMessage('Pov search',text)
        print(text)
        status = False
        
if status:       
    listOfNames = []
    for pov in listOfPov:
        nameOfPov = pov.get('name')
        if not nameOfPov:
            nameOfPov = "NoName"
        idOfpov = pov.get('id')
        str2Display =  idOfpov + ":" +nameOfPov
        listOfNames.append(str2Display)


In [ ]:
listOfNames

### Display the list for selection

In [ ]:
if status:
    povC = uiUtils.chooseWithMenu('PointOfViews',listOfNames)
    if not povC:
        text = "Premise: " + str(myEnv.get('PremiseCode')) + ' No PoV Selected '
        uiUtils.displayMessage('PoV Selection',text)
        print(text)
        status = False
    else:
        tmp = povC.split(':')
        povId = tmp[0]
        if not povId:
            text = "povId: " + str(myEnv.get('PremiseCode')) + ' Error in PoV selection: '+povId
            uiUtils.displayMessage('PoV Selection',text)
            print(text)
            status = False  

### Request device definition

In [ ]:
#%% device information
if status:
    povDetail = povUtils.getPovById(myEnv,povId)
    if not povDetail:
        text = "Premise: " + str(myEnv.get('PremiseCode')) + ' Error getting PoV information: '+ povId
        uiUtils.displayMessage('PoV Selection',text)
        print(text)
        status = False        

### Display device definition

In [ ]:
if status:
    text = "Premise: " + str(myEnv.get('PremiseCode'))

    text  = text +'\n' + '    ID: '+ str(povId)
    povGeneralDefinition = povDetail.get('pov')
    text  = text +'\n' + '        name: '+ str(povGeneralDefinition.get('name'))
    text  = text +'\n' + '        address: '+ str(povGeneralDefinition.get('address'))
    text  = text +'\n' + '        isEnable: '+ str(povGeneralDefinition.get('isEnabled'))

    povType = povDetail.get('povType')
    if povType:
        typeText = "Brand " +povType.get('brand')+ " Model "+povType.get('name')+" Version " +povType.get('version')
        text  = text +'\n' + '        type: '+ typeText
    else:
        text  = text +'\n' + '        type: No PoV Type'
            
    povFixture = povDetail.get('povFixture')
    if povFixture:
        text  = text +'\n' + '        fixture id: '+ povFixture.get('id')
        text  = text +'\n' + '        fixture name: '+ povFixture.get('name')
    else:
        text  = text +'\n' + '        fixture id: No fixture found'
            
    povSettings = povDetail.get('povSettings')


    answer = uiUtils.askQuestionYesNo('PoV move ?',text)

In [ ]:
#%%
if status :
    if answer is None:
        # skip
        print('skip by user')
        status = False
    elif (answer == 'no'):
        #skip
        print('skip by user')

    else :
        fixtureId = povFixture.get('id')
        if not fixtureId:
            text = 'PoV Move: PointOfView not linked to fixture '
            uiUtils.displayMessage('PoV Move:',text)
            print(text)
        else:
            try:
                tmpStatus = povUtils.deleteLinkWithFixture(myEnv, povId, fixtureId)
            except:
                text = 'PoV Move: impossible to delete link PointOfView to fixture '
                uiUtils.displayMessage('PoV Move:',text)
                print(text)
                status = False  

## Link to fixture

In [ ]:
%%capture
if status:
    myEnv = chooseEnv.chooseZone(myEnv);

In [ ]:
%%capture
if status:
    myEnv = chooseEnv.chooseFixture(myEnv, dialogTitle='Select fixture for pov ' + povId)
    fixtureId = myEnv['FixtureId']
    if not fixtureId:
        text = 'PoV Move: no valide fixture selected '
        uiUtils.displayMessage('PoV Move:',text)
        print(text)
        status = False  

In [ ]:
if status:
    try :
        povUtils.linkPovToFixture(myEnv, povId, fixtureId)
    except:
        text = 'PoV Move: impossible to link PointOfView to device '
        uiUtils.displayMessage('PoV Move:',text)
        print(text)
        status = False  